In [1]:
import torch
import numpy as np
import cv2
from utils.training_kits import stdout_to_tqdm, load_pretrained_state

from data import get_dataset
from config import get_config
from utils.visualization_tools import draw_heatmaps, draw_region_maps, draw_point, draw_bbox, draw_text,draw_centermap

cfg, DATASET = get_config("config/freihand/cfg_freihand_hg_ms_att.py")
cfg['batch_size'] = 1
cfg['workers'] = 1

from tensorboardX import SummaryWriter
import torchvision.utils as vutils

new_size = cfg["image_size"][0]
def draw_region_bbox(img, xywhc, color=(0, 0, 255)):
    cx, cy, w, h = xywhc[:4]
    x1, y1, x2, y2 = cx - w / 2, cy - h / 2, cx + w / 2, cy + h / 2
    x1 = int(max(0, x1))
    y1 = int(max(0, y1))
    x2 = int(min(x2, new_size))
    y2 = int(min(y2, new_size))
    img = draw_bbox(img, x1, y1, x2, y2, color=color)
    return img


In [ ]:
#  1、用于可视化用SimDR检测关键点的手部检测
from models.center_simDR import LiteHourglassNet as Network
from utils.result_parser import ResultParser

img, target_x, target_y, target_weight, centermap, centermask, bbox, gt_kpts = \
None, None, None, None, None, None, None, None
class TestPreds:
    def __init__(self, checkpoint="", is_cuda=True, ground_truth=False, exp_name=''):

        print("preparing data...")
        self.dataset, self.test_loader = get_dataset(set_type='test')
        print("done!")

        if is_cuda:
            self.device = torch.device(0)
        else:
            self.device = torch.device('cpu')
        self.model = Network().to(self.device)

        self.ground_truth = ground_truth
        if checkpoint != "":
            print("loading state dict...")
            save_dict = torch.load(checkpoint, map_location=self.device)
            print(f"{save_dict.keys()=}")
            
            state, is_match = load_pretrained_state(self.model.state_dict(),
                                                    save_dict['state_dict'])
            # self.model.load_state_dict(save_dict["state_dict"])
            self.model.load_state_dict(state)
            print(f"done! {is_match=}")
        
        self.writer = SummaryWriter(log_dir='jupyter_log/'+ exp_name)
        self.parser = ResultParser()

    def test(self, n_img=10, show_hms=True, show_kpts=True):
        self.model.eval()
        with torch.no_grad():
            pck = 0
            if n_img == -1:
                n_img = len(self.test_loader)  # -1 整个数据集过一遍
            for i, meta in enumerate(self.test_loader):
                if i > n_img:
                    break
                start = cv2.getTickCount()  # 计时器起点
                img, target_x, target_y, target_weight, centermap, centermask, bbox, gt_kpts = meta
                
                if not self.ground_truth:
                    pred_centermap, pred_x, pred_y = self.model(img.to(self.device))
                else:
                    pred_centermap, pred_x, pred_y = centermap, target_x, target_y
         
                # 结果解析，得到原图关键点和边界框
                pred_kpts, pred_bboxes = self.parser.parse(pred_centermap, pred_x, pred_y)
                ap50, ap = self.parser.evaluate_ap(pred_bboxes, bbox)

                # 画出热图
                if show_hms:
                    out_centermap = draw_centermap(centermap)
                    out_centermap = [torch.tensor(out, dtype=torch.uint8) for out in out_centermap]
                    imgs = torch.stack(out_centermap, dim=0)
                    self.writer.add_image('centermap', imgs, i, dataformats='NHWC')

                # 画出关键点
                if show_kpts:
                    batch_xywh = pred_bboxes[0]
                    if batch_xywh is None:
                        print("没有找到目标框")
                        batch_xywh = [[4, 4, 2, 2, 0]]

                    for image, kpts, xywh in zip(img, pred_kpts, batch_xywh):  
                    # for image, kpts, xywh in zip(img, gt_kpts, batch_xywh):    
                        image = image.permute(1, 2, 0).detach().numpy()
                        m = np.array([0.485, 0.456, 0.406])
                        s = np.array([0.229, 0.224, 0.225])
                        image = image * s + m
                        image *= 255
                        image = image.astype(np.uint8) 
                        
                        # kpts = kpts.squeeze(dim=0).detach().cpu().numpy()
                        kpts = kpts[0].detach().cpu().numpy()
                        # print(f"{image.shape=}")
                        print(f"{bbox=}")
                        print(f"{xywh=}")
                        print(f"{i=} :{kpts=}")
                        print(F"{i=} :{gt_kpts[0, 0]=}")
                        print('*'* 100)
                        
                        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                        image_drawn = draw_point(img=image.copy(), keypoints=kpts)
                        image_drawn = draw_region_bbox(image_drawn, xywh)

                    end = cv2.getTickCount()  # 计时器终点
                    fps = round(cv2.getTickFrequency() / (end - start))
                    text = str(fps) + "fps"
                    img = draw_text(image_drawn, text, (15, 15, 20, 20))
                    # img = img[:,:,::-1]   # BGR to RGB
                    imgs = torch.stack([torch.tensor(image, dtype=torch.uint8),
                                        torch.tensor(image_drawn, dtype=torch.uint8)], dim=0)
                    self.writer.add_images('images', imgs, i, dataformats='NHWC')
                    
            pck = pck / (n_img+1)
            print(f"{n_img=}")
            print(f"{pck=}")
            self.writer.close()
            

In [2]:
#  2、用于可视化用热图检测关键点的手部检测
from models.pose_estimation.pose_hg_ms_att import MultiScaleAttentionHourglass as Network
from utils.evaluation import evaluate_pck, evaluate_ap, get_coordinates_from_heatmap
from utils.result_parser import ResultParser


class TestPreds:
    def __init__(self, checkpoint="", is_cuda=True, ground_truth=False, exp_name='cs'):

        print("preparing data...")
        self.dataset, self.test_loader = get_dataset(cfg, DATASET,
                                                     is_train=False,
                                                     distributed=False)
        print("done!")

        if is_cuda:
            self.device = torch.device(0)
        else:
            self.device = torch.device('cpu')
        self.model = Network(cfg).to(self.device)

        self.ground_truth = ground_truth
        if checkpoint != "":
            print("loading state dict...")
            save_dict = torch.load(checkpoint, map_location=self.device)
            print(f"{save_dict.keys()=}")
            
            state, is_match = load_pretrained_state(self.model.state_dict(),
                                                    save_dict['state_dict'])
            # self.model.load_state_dict(save_dict["state_dict"])
            self.model.load_state_dict(state)
            print(f"done! {is_match=}")
        
        self.result_parser = ResultParser(cfg)
        self.writer = SummaryWriter(log_dir= 'jupyter_log/' + exp_name)

    def test(self, n_img=10, show_hms=True, show_kpts=True, show_cd=False):
        self.model.eval()
        with torch.no_grad():
            pck = 0
            if n_img == -1:
                n_img = len(self.test_loader)  # -1 整个数据集过一遍
            for i, (img, targets, target_weight, bbox, gt_kpts, target_x, target_y) in enumerate(self.test_loader):
                if i > n_img:
                    break
                start = cv2.getTickCount()  # 计时器起点

                if show_cd:  # 显示循环检测的效果
                    img, targets, gt_kpts, target_x, target_y = self.dataset.generate_cd_gt(img, gt_kpts, bbox, target_weight)
                    bbox = torch.tensor([[[img.shape[3] / 2, img.shape[2] / 2,
                                     img.shape[3], img.shape[2]]]], device=bbox.device)

                targets = targets[0]
                region = targets[:, :3]
                hm_kpts = targets[:, 3:].to(self.device)

                if not self.ground_truth:
                    hms_list, pred_x, pred_y  = self.model(img.to(self.device))
                    hm_kpts = hms_list[-1][:, 3:]
                    region = hms_list[-1][:, :3]
                
                pck += evaluate_pck(hm_kpts, targets[:, 3:], bbox, thr=0.2).item()
                print(f"{pck=}")

                # 画出热图
                if show_hms:
                    draw_region_maps(region)
                    draw_heatmaps(hm_kpts)

                # 画出关键点
                if show_kpts:
                    # batch_xywh = cs_from_region_map(batch_region_maps=region, k=1, thr=0.1)
                    _, _, batch_xywh = evaluate_ap(region, bbox, new_size, k=10, iou_thr=0.3)
                    # print(f"{batch_xywh=}")
                    batch_xywh = batch_xywh[0]
                    if batch_xywh is None:
                        print("没有找到目标框")
                        batch_xywh = [[4, 4, 2, 2, 0]]
                        
                    # ! 因为现在是单手，所以特殊处理一下
                    bbox = bbox[0]
                    gt_kpts = gt_kpts[:, 0]

                    first_time_kpt = self.result_parser.get_pred_kpt(hm_kpts)
                    first_time_kpt[:, :, :2] *= torch.tensor([4, 4], device=first_time_kpt.device)
                    pred_bboxes = self.result_parser.get_pred_bbox(region)
                    second_time_kpt = self.result_parser.get_group_keypoints(self.model, img, pred_bboxes, hm_kpts)

                    for image, gk, ftk, stk, xywh, gt_xywh in zip(img, gt_kpts,
                                                          first_time_kpt, second_time_kpt,
                                                          batch_xywh, bbox):    
                        image = image.permute(1, 2, 0).detach().numpy()
                        m = np.array([0.485, 0.456, 0.406])
                        s = np.array([0.229, 0.224, 0.225])
                        image = image * s + m
                        image *= 255
                        image = image.astype(np.uint8) 
                        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                        image_gt = draw_results(image, gk, gt_xywh, gt_xywh)
                        image_drawn1 = draw_results(image, ftk, xywh, gt_xywh)
                        image_drawn2 = draw_results(image, stk, xywh, gt_xywh)

                    end = cv2.getTickCount()  # 计时器终点
                    fps = round(cv2.getTickFrequency() / (end - start))
                    text = str(fps) + "fps"
                    img = draw_text(image_drawn1, text, (15, 15, 20, 20))
                    # img = img[:,:,::-1]   # BGR to RGB
                    imgs = torch.stack([torch.tensor(image, dtype=torch.uint8),
                                        torch.tensor(image_gt, dtype=torch.uint8),
                                        torch.tensor(image_drawn1, dtype=torch.uint8),
                                        torch.tensor(image_drawn2, dtype=torch.uint8),
                                        ], dim=0)
                    # img_grid = vutils.make_grid(imgs, normalize=True, scale_each=True, nrow=2)
                    self.writer.add_images('images', imgs, i, dataformats='NHWC')
                    
            pck = pck / (n_img+1)
            print(f"{n_img=}")
            print(f"{pck=}")
            self.writer.close()

def draw_results(image, kpts, xywh, gt_xywh):
    """在图片上画出预测关键点、预测框和真值框

    Args:
        image (numpy): (h, w, c)
        kpts (tensor): (1, n_joints, 3)
        xywh (tensor): (5,) (cx, cy, w, h, score)
        gt_xywh (tensor): (4,) (cx, cy, w, h)
    """
    kpts = kpts.squeeze(dim=0).detach().cpu().numpy()
    image_drawn = draw_point(img=image.copy(), keypoints=kpts)
    image_drawn = draw_region_bbox(image_drawn, xywh, (255, 0, 0))
    image_drawn = draw_region_bbox(image_drawn, gt_xywh, (0, 255, 0))
    return image_drawn
    

In [3]:
# path = "./checkpoint/MSRB-D-DW-PELEE/1HG-ME-att-c256/2021-12-27/0.981_PCK_47epoch.pt"
path = "checkpoint/final_ME-att/ls/1HG-ME-att-c128-h4-k2-o64-gtbbox-no_augment/2022-03-10/72.566_AP_52epoch.pt"
t = TestPreds(checkpoint=path, is_cuda=False, ground_truth=True, exp_name='ms_freihand/r1')
t.test(n_img=300, show_hms=False, show_kpts=True, show_cd=False)
# tensorboard --samples_per_plugin scalars=100,images=100 --logdir "./jupyter_log/r_gtbbox/"

preparing data...
sample number of testing dataset:  5
done!
loading state dict...
save_dict.keys()=dict_keys(['epoch', 'lr', 'loss', 'mPCK', 'ap', 'state_dict', 'optimizer', 'config'])
done! is_match=False


[ WARN:0@8.927] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/root/data/Dataset/multiscaleimages/00120525.jpg'): can't open/read file: check file path/integrity
[ WARN:0@8.997] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/root/data/Dataset/multiscaleimages/00024296.jpg'): can't open/read file: check file path/integrity
[ WARN:0@9.000] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('/root/data/Dataset/multiscaleimages/00079305.jpg'): can't open/read file: check file path/integrity


error: Caught error in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/root/data/miniconda3/envs/TorchCV/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/root/data/miniconda3/envs/TorchCV/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/root/data/miniconda3/envs/TorchCV/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/root/data/Code/stage-network/data/handset/FreihandDataset.py", line 94, in __getitem__
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

